In [1]:
import pandas as pd
import numpy as np
import gc
import os, shutil
import sys
import matplotlib.pyplot as plt
import open3d as o3d
import transforms3d
from tqdm import tqdm_notebook
import copy

In [2]:
OUTPUT_FOLDER = "6"
DYNAMICSUBFOLDER = "6"

In [3]:
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/"

STATIC_FOLDER = "corr_static"
DYNAMIC_FOLDER = "dynamic"
PAIR_FILE = "pair_with_pose_transform_dynseg_all.csv"


# INPUT_PATH = BASE_PATH
OUTPUT_PATH = os.path.join(BASE_PATH, OUTPUT_FOLDER)

OUTPUT_STATIC_PATH = os.path.join(OUTPUT_PATH, STATIC_FOLDER)
OUTPUT_DYNAMIC_PATH = os.path.join(OUTPUT_PATH, DYNAMIC_FOLDER)

if not os.path.exists(OUTPUT_STATIC_PATH):
    os.makedirs(OUTPUT_STATIC_PATH)
else:
    shutil.rmtree(OUTPUT_STATIC_PATH)
    os.makedirs(OUTPUT_STATIC_PATH)

# os.makedirs(OUTPUT_STATIC_PATH)
# os.makedirs(OUTPUT_DYNAMIC_PATH)

In [4]:
pair_path = os.path.join(BASE_PATH, PAIR_FILE)
df_pair = pd.read_csv(pair_path)
df_pair.shape

(6284, 11)

In [5]:
df_pair.sample()

,static_path,dynamic_path,pose_x,pose_y,pose_yaw,static_x,static_y,static_yaw,dynamic_x,dynamic_y,dynamic_yaw
3203,/home/saby/Projects/ati/data/data/datasets/Car...,/home/saby/Projects/ati/data/data/datasets/Car...,0.12677,-0.382263,-0.258385,92.631294,187.584579,-96.223427,92.758064,187.202316,-96.481812


In [6]:
# Function to get transformation matrix for a given pose
def pose2matrix(translation_list, rotation_angle_list, zoom_list=[1,1,1]):
    # trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom_vec = np.array(zoom_list)
    # transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom_vec)
    transform_mat = transforms3d.affines.compose(translation_list, rot_mat, zoom_list)
    return transform_mat

In [7]:
# def transform_corr_static_pcd(static_pcd, pair_row):
#     # Calculate all the transforms
#     transform_lidar   = pose2matrix([0, 0, 0],
#                                     [0, 0, 90],
#                                     [1, 1, -1])
#     transform_static1  = pose2matrix([pair_row['static_x'], pair_row['static_y'], pair_row['static_z']],
#                                     [pair_row['static_roll'],pair_row['static_pitch'], pair_row['static_yaw']])
#     transform_dynamic1 = pose2matrix([pair_row['dynamic_x'], pair_row['dynamic_y'], pair_row['dynamic_z']],
#                                     [pair_row['dynamic_roll'], pair_row['dynamic_pitch'], pair_row['dynamic_yaw']])
#     transform1 = np.matmul(np.linalg.inv(transform_dynamic1), transform_static1)
    
#     transform_static2  = pose2matrix([0, 0, 0],
#                                     [pair_row['static_roll'],pair_row['static_pitch'], pair_row['static_yaw']])
#     transform_dynamic2 = pose2matrix([0, 0, 0],
#                                     [pair_row['dynamic_roll'], pair_row['dynamic_pitch'], pair_row['dynamic_yaw']])
#     transform2 = np.matmul(np.linalg.inv(transform_dynamic2), transform_static2)
    
#     # Apply all the transforms
#     static_pcd_tmp = copy.deepcopy(static_pcd)
#     static_pcd_tmp.transform(np.linalg.inv(transform_lidar))
#     static_pcd_tmp.transform(np.linalg.inv(transform1))
#     static_pcd_tmp.transform(transform2)
#     static_pcd_tmp.transform(transform2)     # Need to fix this hack
#     static_pcd_tmp.transform(transform_lidar)
    
#     return static_pcd_tmp

def transform_corr_static_pcd(static_pcd, pair_row):
    # Calculate all the transforms
    transform_lidar   = pose2matrix([0, 0, 0],
                                    [0, 0, 90],
                                    [1, 1, -1])
    transform_static1  = pose2matrix([pair_row['static_x'], pair_row['static_y'], 0],
                                    [0,0, pair_row['static_yaw']])
    transform_dynamic1 = pose2matrix([pair_row['dynamic_x'], pair_row['dynamic_y'], 0],
                                    [0, 0, pair_row['dynamic_yaw']])
    transform1 = np.matmul(np.linalg.inv(transform_dynamic1), transform_static1)
    
    transform_static2  = pose2matrix([0, 0, 0],
                                    [0,0, pair_row['static_yaw']])
    transform_dynamic2 = pose2matrix([0, 0, 0],
                                    [0, 0, pair_row['dynamic_yaw']])
    transform2 = np.matmul(np.linalg.inv(transform_dynamic2), transform_static2)
    
    # Apply all the transforms
    static_pcd_tmp = copy.deepcopy(static_pcd)
    static_pcd_tmp.transform(np.linalg.inv(transform_lidar))
    static_pcd_tmp.transform(np.linalg.inv(transform1))
    static_pcd_tmp.transform(transform2)
    static_pcd_tmp.transform(transform2)     # Need to fix this hack
    static_pcd_tmp.transform(transform_lidar)
    
    return static_pcd_tmp

In [8]:
for idx, row in tqdm_notebook(df_pair.iterrows(), total=df_pair.shape[0]):
    if row['dynamic_path'].split("/")[-3] == DYNAMICSUBFOLDER:
        fname = row['dynamic_path'].split("/")[-1]
        fname = str(idx) + ".ply"

        static_src = row['static_path']
        static_dst = os.path.join(OUTPUT_STATIC_PATH, fname)
        static_pcd = o3d.io.read_point_cloud(static_src)
        static_pcd = transform_corr_static_pcd(static_pcd, row)
        o3d.io.write_point_cloud(static_dst, static_pcd, write_ascii=True)
#         shutil.copyfile(static_src, static_dst)

#         dynamic_src = row['dynamic_path']
#         dynamic_dst = os.path.join(OUTPUT_DYNAMIC_PATH, fname)
#         shutil.copyfile(dynamic_src, dynamic_dst)

/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [9]:
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

In [10]:
some_idx = np.random.choice(len(os.listdir(OUTPUT_STATIC_PATH)))
some_fname = str(some_idx) + ".ply"
# some_row = df_pair[(df_pair['dynamic_path'].split("/")[-3] == DYNAMICSUBFOLDER) &\
#             (df_pair['dynamic_path'].split("/")[-1] == some_fname)]

some_static_dst = os.path.join(OUTPUT_STATIC_PATH, some_fname)
some_dynamic_dst = os.path.join(os.path.join(OUTPUT_PATH, "_out"), some_fname)

some_static_pcd = o3d.io.read_point_cloud(some_static_dst)
some_dynamic_pcd = o3d.io.read_point_cloud(some_dynamic_dst)

# print("Pose_diff_x: {} | Pose_diff_y: {} | Pose_diff_yaw: {} |".format(some_row['pose_x'], some_row['pose_y'], some_row['pose_yaw']))
draw_registration_result(some_static_pcd, some_dynamic_pcd, 0, 0, 0)

JVisualizer with 2 geometries